In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-c4zbfcex
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-c4zbfcex
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install datasets transformers torchaudio

In [ ]:
from huggingface_hub import login

login(token="<Your token>")

from datasets import load_dataset

# Load the default configuration of the dataset
dataset = load_dataset("mozilla-foundation/common_voice_17_0", "fa", streaming=True)

# Check the first few samples to identify the 'language' field
for sample in dataset["train"].take(5):
    print(sample)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Reading metadata...: 28893it [00:01, 20671.14it/s]


{'client_id': 'a90b9c9411bc2b1989b02b27b19c25d11be7de25cfb73ac3c3e056b2ece3051d9c49a09f77d0d54b1e786dafb9d67a463a286e1d3dd1c5868bb8f09dc6043b31', 'path': 'fa_train_0/common_voice_fa_33316953.mp3', 'audio': {'path': 'fa_train_0/common_voice_fa_33316953.mp3', 'array': array([ 3.34477024e-28, -6.18466950e-28, -6.81575822e-28, ...,
       -8.99972506e-19,  1.39591030e-18, -1.99899776e-18]), 'sampling_rate': 48000}, 'sentence': 'او مثل آقا رفتار کرد.', 'up_votes': 2, 'down_votes': 0, 'age': '', 'gender': '', 'accent': '', 'locale': 'fa', 'segment': '', 'variant': ''}
{'client_id': 'a90b9c9411bc2b1989b02b27b19c25d11be7de25cfb73ac3c3e056b2ece3051d9c49a09f77d0d54b1e786dafb9d67a463a286e1d3dd1c5868bb8f09dc6043b31', 'path': 'fa_train_0/common_voice_fa_33316954.mp3', 'audio': {'path': 'fa_train_0/common_voice_fa_33316954.mp3', 'array': array([-2.84217094e-14,  1.56319402e-13,  1.31450406e-13, ...,
        2.63489037e-05, -7.29196472e-08, -8.64365666e-06]), 'sampling_rate': 48000}, 'sentence': 'زیب

In [ ]:
train_dataset = dataset["train"]
validation_dataset = dataset["validation"]

# Preview a few samples
for sample in train_dataset.take(5):
    print(sample)

Reading metadata...: 28893it [00:01, 27169.88it/s]


{'client_id': 'a90b9c9411bc2b1989b02b27b19c25d11be7de25cfb73ac3c3e056b2ece3051d9c49a09f77d0d54b1e786dafb9d67a463a286e1d3dd1c5868bb8f09dc6043b31', 'path': 'fa_train_0/common_voice_fa_33316953.mp3', 'audio': {'path': 'fa_train_0/common_voice_fa_33316953.mp3', 'array': array([ 3.34477024e-28, -6.18466950e-28, -6.81575822e-28, ...,
       -8.99972506e-19,  1.39591030e-18, -1.99899776e-18]), 'sampling_rate': 48000}, 'sentence': 'او مثل آقا رفتار کرد.', 'up_votes': 2, 'down_votes': 0, 'age': '', 'gender': '', 'accent': '', 'locale': 'fa', 'segment': '', 'variant': ''}
{'client_id': 'a90b9c9411bc2b1989b02b27b19c25d11be7de25cfb73ac3c3e056b2ece3051d9c49a09f77d0d54b1e786dafb9d67a463a286e1d3dd1c5868bb8f09dc6043b31', 'path': 'fa_train_0/common_voice_fa_33316954.mp3', 'audio': {'path': 'fa_train_0/common_voice_fa_33316954.mp3', 'array': array([-2.84217094e-14,  1.56319402e-13,  1.31450406e-13, ...,
        2.63489037e-05, -7.29196472e-08, -8.64365666e-06]), 'sampling_rate': 48000}, 'sentence': 'زیب

In [ ]:
from datasets import Audio

# Cast the "audio" column to download the files locally
train_dataset = train_dataset.cast_column("audio", Audio())
validation_dataset = validation_dataset.cast_column("audio", Audio())


In [ ]:
!pip install torch torchaudio

In [ ]:
def preprocess_and_save(dataset, split_name, chunk_size=100):
    import torchaudio

    batch = []
    chunk_index = 0  # To keep track of the current chunk index

    for i, example in enumerate(dataset):
        # Access the audio path
        audio = example["audio"]["array"]  # Extract raw audio data
        transcript = example["sentence"]

        batch.append({"audio": audio, "transcript": transcript})

        # Save the chunk when the batch reaches the specified size
        if (i + 1) % chunk_size == 0:
            chunk_path = f"processed/{split_name}/chunk_{chunk_index}.pt"
            torch.save(batch, chunk_path)
            batch = []  # Clear the batch to save memory
            chunk_index += 1

    # Save any remaining data
    if batch:
        chunk_path = f"processed/{split_name}/chunk_{chunk_index}.pt"
        torch.save(batch, chunk_path)

# Process and save train and validation datasets
preprocess_and_save(train_dataset, "train")
preprocess_and_save(validation_dataset, "validation")


In [ ]:
import os

train_chunks = os.listdir("processed/train")
validation_chunks = os.listdir("processed/validation")
print(f"Train chunks: {train_chunks}")
print(f"Validation chunks: {validation_chunks}")


Train chunks: ['chunk_35.pt', 'chunk_244.pt', 'chunk_132.pt', 'chunk_187.pt', 'chunk_173.pt', 'chunk_77.pt', 'chunk_163.pt', 'chunk_59.pt', 'chunk_148.pt', 'chunk_169.pt', 'chunk_44.pt', 'chunk_123.pt', 'chunk_204.pt', 'chunk_258.pt', 'chunk_23.pt', 'chunk_179.pt', 'chunk_221.pt', 'chunk_8.pt', 'chunk_3.pt', 'chunk_33.pt', 'chunk_282.pt', 'chunk_5.pt', 'chunk_227.pt', 'chunk_6.pt', 'chunk_25.pt', 'chunk_127.pt', 'chunk_256.pt', 'chunk_269.pt', 'chunk_0.pt', 'chunk_69.pt', 'chunk_255.pt', 'chunk_124.pt', 'chunk_245.pt', 'chunk_199.pt', 'chunk_195.pt', 'chunk_128.pt', 'chunk_84.pt', 'chunk_203.pt', 'chunk_215.pt', 'chunk_19.pt', 'chunk_152.pt', 'chunk_97.pt', 'chunk_78.pt', 'chunk_32.pt', 'chunk_263.pt', 'chunk_39.pt', 'chunk_250.pt', 'chunk_28.pt', 'chunk_164.pt', 'chunk_65.pt', 'chunk_36.pt', 'chunk_89.pt', 'chunk_180.pt', 'chunk_24.pt', 'chunk_12.pt', 'chunk_225.pt', 'chunk_175.pt', 'chunk_57.pt', 'chunk_40.pt', 'chunk_133.pt', 'chunk_1.pt', 'chunk_144.pt', 'chunk_117.pt', 'chunk_262.

In [ ]:
def load_chunk(split_name, chunk_index):
    import torch

    chunk_path = f"processed/{split_name}/chunk_{chunk_index}.pt"
    return torch.load(chunk_path)  # Returns a list of dictionaries with 'audio' and 'transcript'


In [ ]:
# Load the first chunk from the training set
train_chunk_0 = load_chunk("train", 0)
print(f"Loaded {len(train_chunk_0)} examples from the first training chunk.")


<ipython-input-8-7ef96ad826c7>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(chunk_path)  # Returns a list of dictionaries with 'audio' and 'transcript'

Loaded 100 examples from the first training chunk.


In [ ]:
def stream_batches(split_name, total_chunks, batch_size=4):
    """
    Generator that streams batches from preprocessed chunks.
    """
    for chunk_index in range(total_chunks):
        chunk = load_chunk(split_name, chunk_index)
        for i in range(0, len(chunk), batch_size):
            batch = chunk[i:i+batch_size]
            yield batch

# Example: Stream batches from the train set
train_batches = stream_batches("train", total_chunks=len(train_chunks), batch_size=4)
for batch in train_batches:
    print(batch)
    break  # Print one batch


<ipython-input-8-7ef96ad826c7>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(chunk_path)  # Returns a list of dictionaries with 'audio' and 'transcript'

[{'audio': array([0., 0., 0., ..., 0., 0., 0.]), 'transcript': 'او مثل آقا رفتار کرد.'}, {'audio': array([ 0.00000000e+00,  1.32521056e-13, -7.95415166e-14, ...,
        3.36417179e-05,  2.67675605e-05, -8.01658098e-06]), 'transcript': 'زیبایی او را نمیتوانم وصف کنم.'}, {'audio': array([ 0.00000000e+00, -5.71812630e-15, -3.87703074e-14, ...,
        5.19313403e-07,  3.44763549e-07,  1.15781017e-06]), 'transcript': 'بدون قید و شرط'}, {'audio': array([ 0.00000000e+00,  1.23193534e-12,  2.48242160e-12, ...,
       -1.42956735e-04, -1.15126917e-04, -1.31362525e-04]), 'transcript': 'این وظیفهی تو است، سعی نکن آن را به من محول کنی!'}]


In [ ]:
from torch.utils.data import Dataset

class ChunkedDataset(Dataset):
    def __init__(self, split_name, total_chunks):
        self.split_name = split_name
        self.total_chunks = total_chunks
        self.current_chunk = None
        self.chunk_index = -1

    def load_next_chunk(self):
        self.chunk_index += 1
        if self.chunk_index < self.total_chunks:
            self.current_chunk = load_chunk(self.split_name, self.chunk_index)
        else:
            self.current_chunk = None

    def __len__(self):
        return self.total_chunks * len(self.current_chunk)

    def __getitem__(self, idx):
        chunk_idx = idx // len(self.current_chunk)
        within_chunk_idx = idx % len(self.current_chunk)
        if chunk_idx != self.chunk_index:
            self.chunk_index = chunk_idx
            self.current_chunk = load_chunk(self.split_name, self.chunk_index)
        return self.current_chunk[within_chunk_idx]

# Example: Create train dataset
train_dataset = ChunkedDataset("train", total_chunks=len(train_chunks))


In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

# Load the Whisper model
model_name = "openai/whisper-large-v3"
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# Load the processor (tokenizer + feature extractor)
processor = WhisperProcessor.from_pretrained(model_name)
tokenizer = processor.tokenizer


In [20]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper_fine_tuned",    # Directory to save model checkpoints
    per_device_train_batch_size=4,       # Adjust batch size to fit in memory
    per_device_eval_batch_size=4,
    learning_rate=5e-5,                  # Learning rate
    num_train_epochs=3,                  # Number of training epochs
    evaluation_strategy="epoch",         # Evaluate at the end of each epoch
    save_strategy="epoch",               # Save model checkpoints
    logging_steps=1,
    max_steps=1000,
    logging_dir="./logs",                # Directory for logs
    predict_with_generate=True,          # Generate text for evaluation
    fp16=True,                           # Enable mixed precision training (if supported)
    save_total_limit=2,                  # Limit the number of checkpoints
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(processor.tokenizer, model=model)

In [22]:
from torch.utils.data import Dataset
import torch

class CustomDataset(Dataset):
    def __init__(self, chunks_path, total_chunks):
        self.chunks_path = chunks_path
        self.total_chunks = total_chunks

    def __len__(self):
        return sum(len(torch.load(f"{self.chunks_path}/chunk_{i}.pt")) for i in range(self.total_chunks))

    def __getitem__(self, idx):
        chunk_idx = idx // len(torch.load(f"{self.chunks_path}/chunk_0.pt"))  # Determine which chunk
        local_idx = idx % len(torch.load(f"{self.chunks_path}/chunk_0.pt"))  # Index within the chunk
        chunk = torch.load(f"{self.chunks_path}/chunk_{chunk_idx}.pt")
        return chunk[local_idx]


In [23]:
train_dataset = CustomDataset("processed/train", total_chunks=len(train_chunks))
validation_dataset = CustomDataset("processed/validation", total_chunks=len(validation_chunks))

In [24]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=ChunkedDataset("validation", total_chunks=len(validation_chunks)),
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-24-08c8795bec9e>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
<ipython-input-22-83299940537d>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full c

In [25]:
from transformers import TrainerCallback
import time

class TimeEstimatorCallback(TrainerCallback):
    def __init__(self, total_steps):
        self.total_steps = total_steps
        self.start_time = time.time()

    def on_log(self, args, state, control, **kwargs):
        elapsed_time = time.time() - self.start_time
        steps_done = state.global_step
        if steps_done > 0:
            remaining_steps = self.total_steps - steps_done
            time_per_step = elapsed_time / steps_done
            remaining_time = remaining_steps * time_per_step
            print(f"Time remaining: {remaining_time / 60:.2f} minutes")

# Add total steps calculation
train_steps_per_epoch = len(train_dataset) // training_args.per_device_train_batch_size
total_train_steps = train_steps_per_epoch * training_args.num_train_epochs

# Attach callback
trainer.add_callback(TimeEstimatorCallback(total_train_steps))

<ipython-input-22-83299940537d>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return sum(len(torch.load(f"{self.chunks_path}/chunk_{i}.pt")) for i in range(self.total_ch

In [ ]:
trainer.train()

<ipython-input-22-83299940537d>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return sum(len(torch.load(f"{self.chunks_path}/chunk_{i}.pt")) for i in range(self.total_ch